In [1]:
import pandas as pd
import numpy as np

In [204]:
df = pd.read_csv("data/premier_league.csv")
#df = pd.read_csv('data/f1.csv')

In [205]:
df

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,09/08/2019,20:00,Liverpool,Norwich,4,1,H,4,0,...,3.43,-2.25,1.91,1.99,1.94,1.98,1.99,2.07,1.90,1.99
1,E0,10/08/2019,12:30,West Ham,Man City,0,5,A,0,1,...,2.91,1.75,1.95,1.95,1.96,1.97,2.07,1.98,1.97,1.92
2,E0,10/08/2019,15:00,Bournemouth,Sheffield United,1,1,D,0,0,...,1.92,-0.50,1.95,1.95,1.98,1.95,2.00,1.96,1.96,1.92
3,E0,10/08/2019,15:00,Burnley,Southampton,3,0,H,0,0,...,1.71,0.00,1.87,2.03,1.89,2.03,1.90,2.07,1.86,2.02
4,E0,10/08/2019,15:00,Crystal Palace,Everton,0,0,D,0,0,...,1.71,0.25,1.82,2.08,1.97,1.96,2.03,2.08,1.96,1.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,E0,07/03/2020,15:00,Wolves,Brighton,0,0,D,0,0,...,1.74,-0.75,1.98,1.92,2.01,1.92,2.03,1.99,1.94,1.93
284,E0,07/03/2020,17:30,Burnley,Tottenham,1,1,D,1,0,...,1.74,0.00,1.95,1.95,1.96,1.97,2.00,2.01,1.94,1.92
285,E0,08/03/2020,14:00,Chelsea,Everton,4,0,H,2,0,...,2.13,-0.50,1.85,2.05,1.88,2.04,2.00,2.11,1.86,2.01
286,E0,08/03/2020,16:30,Man United,Man City,2,0,H,1,0,...,1.97,0.75,1.87,2.03,1.85,2.07,1.91,2.11,1.87,2.01


In [206]:
def prediction (odds1,odds2,odds3):
    if odds1<odds3 and odds1<odds2:
        predict = 0
    elif odds3<odds1 and odds3<odds2:
        predict = 1
    elif odds2< odds1 and odds2<odds3:
        predict=2
    else:
        predict=3
    return predict

def win_draw_loose(result):
    if result == 'H':
        points = 3
    elif result == 'D':
        points = 1
    elif result == 'A':
        points =0
    return points

def bookeh_right(result,prediction):
    if result == 3 and prediction == 0:
        is_right = True
    elif result == 0 and prediction == 1:
        is_right = True
    elif result == 1 and prediction ==2:
        is_right = True
    else:
        is_right = False
    return is_right

In [207]:
true = np.array([])
false = np.array([])
list_bookeh = np.array(['B365','BW','IW','PS','VC','WH'])
list_league = np.array(['E0','F1'])
df['points'] = df['FTR'].apply(win_draw_loose)
for i in range(len(list_bookeh)):
    y = list_bookeh[i]
    df[y+'pred'] = df.apply(lambda x: prediction(x[y+'H'], x[y+'D'],x[y+'A']), axis=1)
    df[y+'right'] = df.apply(lambda x: bookeh_right(x['points'],x[y+'pred']),axis=1)
    true = np.append(true,df[y+'right'].value_counts()[0])
    false = np.append(false,df[y+'right'].value_counts()[1])
d = {'bookeh':list_bookeh,'true': true,'false':false}
prediction_df = pd.DataFrame(data=d)
prediction_df['precision']= prediction_df.true/(prediction_df.true + prediction_df.false)
prediction_df.sort_values(by='precision',ascending=False)

,bookeh,true,false,precision
3,PS,138.0,150.0,0.479167
1,BW,137.0,151.0,0.475694
0,B365,136.0,152.0,0.472222
2,IW,136.0,152.0,0.472222
4,VC,136.0,152.0,0.472222
5,WH,136.0,152.0,0.472222


In [209]:
df.groupby('HomeTeam').sum().sort_values(by='points',ascending=False)['points']

HomeTeam
Liverpool           45
Leicester           30
Man City            29
Man United          29
Tottenham           26
Arsenal             26
Everton             25
Chelsea             24
Sheffield United    24
Burnley             23
Crystal Palace      22
Wolves              22
Newcastle           21
Brighton            18
Aston Villa         17
Bournemouth         17
Watford             17
Norwich             15
West Ham            15
Southampton         14
Name: points, dtype: int64